##Мутации

В таблицу гена, добавьте следующую информацию: количество rs (SNP) для избранного гена.

In [143]:
!python -m pip install biopython

In [144]:
import sys
from Bio.Seq import MutableSeq
import pandas as pd
import numpy as np
from Bio.Seq import Seq
import time
from Bio import Entrez

In [145]:
genes_table=pd.read_csv('genes_table.csv')

In [146]:
genes_table

,Unnamed: 0.1,Unnamed: 0,geneid,gene_symbol,gene_desc,locus,chromosome,chromosome_desc,seq_id,seq_start,seq_end,strand,seq,gene_syn,protein,q_transcripts,len_long_prot,seq_dna,snps_id,q_snps
0,0,0,7157,TP53,tumor protein p53,17p13.1,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,7668421,7687489,minus,GGCAGCAAAGTTTTATTGTAAAATAAGAGATCGATATAAAAATGGG...,"P53, BCC7, LFS1, BMFS5, TRP53",SKV*SHRPGSR*LLGSGDTLRSGWERAFHDGDTLPWIG*APD*T**...,25,41,ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACC...,"530941076, 201744589, 138729528, 28934576, 115...",5
1,1,1,2771,GNAI2,G protein subunit alpha i2,3p21.31,NC_000003,"NC_000003.12 Homo sapiens chromosome 3, GRCh38...",568815595,50227068,50263357,plus,CATCACCGTCTAATCTCTGCTGTGAAGTGGAAGCGCGAGAAGGAGG...,"GIP, HG1C, GNAI2B, H_LUCA15.1, H_LUCA16.1",HHRLISAVKWKREKEGASHDGGCEDARLDEAERRVSLGTF*G*ADG...,7,47,TAGAGCCTGCAGGCAAGCATGTCACAGCCCACTGAGAGGGTCTTCA...,NaN,0
2,2,2,5573,PRKAR1A,protein kinase cAMP-dependent type I regulator...,17q24.2,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,68413623,68551315,plus,AGCTGTGGTGGGCTCCACCCAGTTCGAGCTTCCCGGCTGCTTTGGT...,"CAR, CNC, CNC1, PKR1, TSE1, ADOHR, PPNAD1, PRK...",SCGGLHPVRASRLLWLPNQAWAMAGAPPPASLPPCSLISDCCASNQ...,11,107,GAGCTGTCGCCTAGCCGCTATCGCAGAGTGGAGCGGGGCTGGGAGC...,139620139,1


## Мутации будум исследовать для гена в первой строки

In [147]:
seq_sl=genes_table['seq_dna'][0]
seq_sl

'ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGCGCGGACGCGGGTGCCGGGCGGGGGTGTGGAATCAACCCACAGCTGCACAGGGCAGGTCTTGGCCAGTTGGCAAAACATCTTGTTGAGGGCAGGGGAGTACTGTAG'

Проверяем делится ли длинна гена на 3

In [148]:
len(seq_sl)%3

0

In [149]:
def show_result(normal_seq_protein, mutable_seq_protein):
    print('normal_seq_protein ', normal_seq_protein)
    print('mutable_seq_protein ', mutable_seq_protein)

    list_normal_seq_protein=list(normal_seq_protein)
    list_mutable_seq_protein=list(mutable_seq_protein)

    no_differ=True
    for i, (a, b) in enumerate(zip(list_normal_seq_protein, list_mutable_seq_protein)):
      if a!=b:
          print(f'Различие в протеинах на позиции {i}: {a} != {b}')
          no_differ=False

    if no_differ:
        print('Протеины одинаковы')


#Мутация: Замена нуклида (substitution)

In [150]:
def get_mutation_substitution(seq_sl, title_mutation='Мутация: Замена нуклида (substitution)'):
    number_first=list(range(0,len(seq_sl), 3))
    while True:
        rand_codon_num = np.random.choice(number_first)
        selected_codon = seq_sl[rand_codon_num:rand_codon_num+3]
        if selected_codon in ['TAA', 'TAG', 'TGA']:
            continue  # это стоп-кодон — выбираем заново
        else:
            chosen_position = rand_codon_num
            break

    while True:
        rand_nucleotide = np.random.choice(['A', 'T', 'G', 'C'])
        if rand_nucleotide == seq_sl[chosen_position]:
            continue  # не менять на ту же букву
        else:
            break




    mutable_seq = MutableSeq(seq_sl)
    mutable_seq[chosen_position] = rand_nucleotide

    mutable_seq_protein=mutable_seq.transcribe().translate()
    normal_seq_protein=Seq(seq_sl).transcribe().translate()

    print(f'{title_mutation}: {seq_sl[chosen_position]}->{rand_nucleotide}  , на позиции {chosen_position}\n')

    show_result(normal_seq_protein, mutable_seq_protein)


    return mutable_seq_protein, normal_seq_protein, mutable_seq, chosen_position, rand_nucleotide





In [151]:
get_mutation_substitution(seq_sl)

Мутация: Замена нуклида (substitution): C->G  , на позиции 36

normal_seq_protein  IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKHLVEGRGVL*
mutable_seq_protein  IAI*AALMVGAAAHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKHLVEGRGVL*
Различие в протеинах на позиции 12: P != A


(MutableSeq('IAI*AALMVGAAAHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKH...VL*'),
 Seq('IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKH...VL*'),
 MutableSeq('ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGGCTCACAACCTCCGTCAT...TAG'),
 np.int64(36),
 np.str_('G'))

In [152]:
def calculate_translate(mutable_seq, seq_sl):
      mutable_seq_protein=Seq(mutable_seq).transcribe().translate()
      normal_seq_protein=Seq(seq_sl).transcribe().translate()
      show_result(normal_seq_protein, mutable_seq_protein)
      return mutable_seq_protein, normal_seq_protein

#Мутация: Вставка нуклида (insertion)'

In [153]:
def get_mutation_insertion(seq_sl, title_mutation='Мутация: Вставка нуклида (insertion)'):
  length=len(seq_sl)-2
  position=np.random.randint(0,length)

  inserted_nucl=np.random.choice(['A', 'T', 'G', 'C'])
  mutable_seq=str(seq_sl[:position]+inserted_nucl+seq_sl[position:])
  print(f'{title_mutation}: {inserted_nucl}  , на позицию {position}\n')

  mutable_seq_protein, normal_seq_protein=calculate_translate(mutable_seq, seq_sl)

  return mutable_seq_protein, normal_seq_protein, mutable_seq, position, inserted_nucl


In [154]:
get_mutation_insertion(seq_sl)

Мутация: Вставка нуклида (insertion): A  , на позицию 152

normal_seq_protein  IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKHLVEGRGVL*
mutable_seq_protein  IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLGKTSC*GQGSTV
Различие в протеинах на позиции 51: A != G
Различие в протеинах на позиции 53: H != T
Различие в протеинах на позиции 54: L != S
Различие в протеинах на позиции 55: V != C
Различие в протеинах на позиции 56: E != *
Различие в протеинах на позиции 58: R != Q
Различие в протеинах на позиции 60: V != S
Различие в протеинах на позиции 61: L != T
Различие в протеинах на позиции 62: * != V


/usr/local/lib/python3.11/dist-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


(Seq('IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLGKT...STV'),
 Seq('IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKH...VL*'),
 'ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGCGCGGACGCGGGTGCCGGGCGGGGGTGTGGAATCAACCCACAGCTGCACAGGGCAGGTCTTGGCCAGTTAGGCAAAACATCTTGTTGAGGGCAGGGGAGTACTGTAG',
 152,
 np.str_('A'))

#Мутация: Удаление нуклида (deletion)

In [155]:
def get_mutation_deletion(seq_sl, title_mutation='Мутация: Удаление нуклида (deletion)'):
  length=len(seq_sl)-2
  position=np.random.randint(0,length)

  mutable_seq=str(seq_sl[:position]+seq_sl[position+1:])

  print(f'{title_mutation}: {seq_sl[position]}  , на позиции {position}\n')
  mutable_seq_protein, normal_seq_protein=calculate_translate(mutable_seq, seq_sl)

  return mutable_seq_protein, normal_seq_protein, mutable_seq, position


In [156]:
get_mutation_deletion(seq_sl)

Мутация: Удаление нуклида (deletion): G  , на позиции 3

normal_seq_protein  IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKHLVEGRGVL*
mutable_seq_protein  ILSEQRSWWGQRLTTSVMCCDCL*MAMARTRVPGGGVESTHSCTGQVLASWQNILLRAGEYC
Различие в протеинах на позиции 1: A != L
Различие в протеинах на позиции 2: I != S
Различие в протеинах на позиции 3: * != E
Различие в протеинах на позиции 4: A != Q
Различие в протеинах на позиции 5: A != R
Различие в протеинах на позиции 6: L != S
Различие в протеинах на позиции 7: M != W
Различие в протеинах на позиции 8: V != W
Различие в протеинах на позиции 10: A != Q
Различие в протеинах на позиции 11: A != R
Различие в протеинах на позиции 12: P != L
Различие в протеинах на позиции 13: H != T
Различие в протеинах на позиции 14: N != T
Различие в протеинах на позиции 15: L != S
Различие в протеинах на позиции 16: R != V
Различие в протеинах на позиции 17: H != M
Различие в протеинах на позиции 18: V != C
Различие в протеинах на позиции 19: L != C
Различие 

(Seq('ILSEQRSWWGQRLTTSVMCCDCL*MAMARTRVPGGGVESTHSCTGQVLASWQNI...EYC'),
 Seq('IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKH...VL*'),
 'ATCCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGCGCGGACGCGGGTGCCGGGCGGGGGTGTGGAATCAACCCACAGCTGCACAGGGCAGGTCTTGGCCAGTTGGCAAAACATCTTGTTGAGGGCAGGGGAGTACTGTAG',
 3)

# Мутация Дупликация / Повтор (Duplication)

In [157]:
def get_mutation_duplication(seq_sl, title_mutation='Дупликация / Повтор (Duplication)'):
  length = len(seq_sl) - 3
  position = np.random.randint(0, length)
  duplicated = seq_sl[position:position+3]

    # Вставляем дубликат сразу после оригинального кодона
  mutable_seq = seq_sl[:position+3] + duplicated + seq_sl[position+3:]

  print(f'{title_mutation}: {duplicated}  , на позиции {position}\n')
  mutable_seq_protein, normal_seq_protein=calculate_translate(mutable_seq, seq_sl)

  return mutable_seq_protein, normal_seq_protein, mutable_seq, position, duplicated


In [158]:
get_mutation_duplication(seq_sl)

Дупликация / Повтор (Duplication): TCT  , на позиции 7

normal_seq_protein  IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKHLVEGRGVL*
mutable_seq_protein  IAIF*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKHLVEGRGVL*
Различие в протеинах на позиции 3: * != F
Различие в протеинах на позиции 4: A != *
Различие в протеинах на позиции 6: L != A
Различие в протеинах на позиции 7: M != L
Различие в протеинах на позиции 8: V != M
Различие в протеинах на позиции 9: G != V
Различие в протеинах на позиции 10: A != G
Различие в протеинах на позиции 12: P != A
Различие в протеинах на позиции 13: H != P
Различие в протеинах на позиции 14: N != H
Различие в протеинах на позиции 15: L != N
Различие в протеинах на позиции 16: R != L
Различие в протеинах на позиции 17: H != R
Различие в протеинах на позиции 18: V != H
Различие в протеинах на позиции 19: L != V
Различие в протеинах на позиции 20: * != L
Различие в протеинах на позиции 21: L != *
Различие в протеинах на позиции 23: V != L
Различ

(Seq('IAIF*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAK...VL*'),
 Seq('IAI*AALMVGAAPHNLRHVL*LLVDGHGADAGAGRGCGINPQLHRAGLGQLAKH...VL*'),
 'ATCGCTATCTTCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACCTCCGTCATGTGCTGTGACTGCTTGTAGATGGCCATGGCGCGGACGCGGGTGCCGGGCGGGGGTGTGGAATCAACCCACAGCTGCACAGGGCAGGTCTTGGCCAGTTGGCAAAACATCTTGTTGAGGGCAGGGGAGTACTGTAG',
 7,
 'TCT')

## Получение данных о мутациях SNP по каждому гену из таблицы

In [159]:
def get_snp(gene_symbol):
    Entrez.email = "dbsnp-user@nih.gov" # provide your user email
    # RECOMMENDED: apply for API key from NCBI (https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/).
    # 10 queries per second with a valid API key, otherwise 3 queries per seconds are allowed for 'None'
    Entrez.api_key = None

    # dbSNP supported query terms (https://www.ncbi.nlm.nih.gov/snp/docs/entrez_help/) can be build and test online using web query builder (https://www.ncbi.nlm.nih.gov/snp/advanced)
    # esearch handle
    eShandle = Entrez.esearch(db="snp",  # search dbSNP
                              #complex query for missense and pathogenic variants in LPL gene with global MAF betweeen 0 and 0.01.
                              term=f'{gene_symbol}[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class] AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])',
                              usehistory="y", #cache result on server for download in batches
                              retmax=100 # return 20 RSID max
                            )
    #get esearch result
    eSresult = Entrez.read(eShandle)
    # review results
    for k in eSresult:
        print (k, ":", eSresult[k])

    #Output: Web environment (&WebEnv) and query key (&query_key) parameters specifying the location on the Entrez history server of the list of UIDs matching the Entrez query
    #https://www.ncbi.nlm.nih.gov/books/NBK25500/#chapter1.Storing_Search_Results

    # get result RSIDs list 'Idlist'
    # total rs count
    rslist = eSresult['IdList']
    # retmax = 20 so print only 20 RSIDs
    # additional results can be retrieved by batches
    # download in batches example http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139 or see below.
    print ("RSIDs:")
    for rs in rslist:
        print(rs)

    return rslist, eSresult


In [160]:
    # get the WebEnv session cookie, and the QueryKey:
def get_snp_info(eSresult=None):
      webenv = eSresult["WebEnv"]
      query_key = eSresult["QueryKey"]
      total_count = int(eSresult["Count"])
      query_key = eSresult["QueryKey"]
      retmax = 2 # return 2 rs per batch example
      # sample codes adopted with modifications from http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc139.
      fetch_count = 0
      for start in range(0, total_count, retmax):
          end = min(total_count, start+retmax)
          print("Going to download record %i to %i" % (start+1, end))
          attempt = 0
          #fetch_count += 1
          while (attempt < 3):
              attempt += 1
              try:
                  fetch_handle = Entrez.efetch(db="snp",
                                              #rettype="uilist", #available types [uilist | xml (use retmode=xml))
                                              retmode="xml",
                                              retstart=start,
                                              retmax=retmax,
                                              webenv=webenv,
                                              query_key=query_key )
              except HTTPError as err:
                  if 500 <= err.code <= 599:
                      print("Received error from server %s" % err)
                      print("Attempt %i of 3" % attempt)
                      time.sleep(15)
                  else:
                      raise
          if (fetch_handle):
              #print(fetch_handle)
              data = fetch_handle.read()
              print(data)
              fetch_handle.close()


In [161]:
def get_snp_db(data_table=genes_table):
      final_db = genes_table
      final_db[['snps_id', 'q_snps']]=None


      for idx, row in final_db.iterrows():
            gene_symbol=row['gene_symbol']
            rslist, eSresult=get_snp(gene_symbol)
            final_db.loc[idx,'snps_id']=', '.join(rslist)
            final_db.loc[idx, 'q_snps'] = len(rslist)
            print(f'\n**Информация по SNP для {gene_symbol}**')
            get_snp_info(eSresult)

      return final_db

In [162]:
genes_table=get_snp_db(genes_table)

Count : 5
RetMax : 5
RetStart : 0
QueryKey : 1
WebEnv : MCID_681612c6a5563ea1360a25df
IdList : ['530941076', '201744589', '138729528', '28934576', '11540654']
TranslationSet : []
TranslationStack : [{'Term': 'TP53[All Fields]', 'Field': 'All Fields', 'Count': '12527', 'Explode': 'N'}, {'Term': 'pathogenic[Clinical_Significance]', 'Field': 'Clinical_Significance', 'Count': '180688', 'Explode': 'N'}, 'AND', {'Term': 'missense variant[Function_Class]', 'Field': 'Function_Class', 'Count': '14319754', 'Explode': 'N'}, 'AND', {'Term': '00000.0000[GLOBAL_MAF]', 'Field': 'GLOBAL_MAF', 'Count': '0', 'Explode': 'N'}, {'Term': '00000.0100[GLOBAL_MAF]', 'Field': 'GLOBAL_MAF', 'Count': '0', 'Explode': 'N'}, 'RANGE', 'GROUP', 'AND']
QueryTranslation : TP53[All Fields] AND pathogenic[Clinical_Significance] AND missense variant[Function_Class] AND (00000.0000[GLOBAL_MAF] : 00000.0100[GLOBAL_MAF])
RSIDs:
530941076
201744589
138729528
28934576
11540654

**Информация по SNP для TP53**
Going to download r

In [163]:
display(genes_table)

,Unnamed: 0.1,Unnamed: 0,geneid,gene_symbol,gene_desc,locus,chromosome,chromosome_desc,seq_id,seq_start,seq_end,strand,seq,gene_syn,protein,q_transcripts,len_long_prot,seq_dna,snps_id,q_snps
0,0,0,7157,TP53,tumor protein p53,17p13.1,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,7668421,7687489,minus,GGCAGCAAAGTTTTATTGTAAAATAAGAGATCGATATAAAAATGGG...,"P53, BCC7, LFS1, BMFS5, TRP53",SKV*SHRPGSR*LLGSGDTLRSGWERAFHDGDTLPWIG*APD*T**...,25,41,ATCGCTATCTGAGCAGCGCTCATGGTGGGGGCAGCGCCTCACAACC...,"530941076, 201744589, 138729528, 28934576, 115...",5
1,1,1,2771,GNAI2,G protein subunit alpha i2,3p21.31,NC_000003,"NC_000003.12 Homo sapiens chromosome 3, GRCh38...",568815595,50227068,50263357,plus,CATCACCGTCTAATCTCTGCTGTGAAGTGGAAGCGCGAGAAGGAGG...,"GIP, HG1C, GNAI2B, H_LUCA15.1, H_LUCA16.1",HHRLISAVKWKREKEGASHDGGCEDARLDEAERRVSLGTF*G*ADG...,7,47,TAGAGCCTGCAGGCAAGCATGTCACAGCCCACTGAGAGGGTCTTCA...,,0
2,2,2,5573,PRKAR1A,protein kinase cAMP-dependent type I regulator...,17q24.2,NC_000017,"NC_000017.11 Homo sapiens chromosome 17, GRCh3...",568815581,68413623,68551315,plus,AGCTGTGGTGGGCTCCACCCAGTTCGAGCTTCCCGGCTGCTTTGGT...,"CAR, CNC, CNC1, PKR1, TSE1, ADOHR, PPNAD1, PRK...",SCGGLHPVRASRLLWLPNQAWAMAGAPPPASLPPCSLISDCCASNQ...,11,107,GAGCTGTCGCCTAGCCGCTATCGCAGAGTGGAGCGGGGCTGGGAGC...,139620139,1


Ген GNAI2 не попадает под жёсткий фильтр (патогенные, missense, редкие) в методе get_snp. Поэтому RSIDs не нашлось.

In [164]:
genes_table.to_csv('genes_table.csv')